In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# ML
from kernels import gaussian_kernel
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import KernelCenterer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer

# Utilities
import h5py
from tempfile import mkdtemp
from shutil import rmtree
from tqdm.notebook import tqdm
from tools import load_json, save_json
import project_utils as utils
from copy import deepcopy

In [3]:
# Load train and test sets
train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/train.idxs', dtype=int)
cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_5.idxs', dtype=int)

In [4]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [5]:
# Make directory in which to store models
os.makedirs('../Processed_Data/Models', exist_ok=True)

In [6]:
# Set property names for loading
property_names = ['volumes', 'energies']

# Load structure properties
structure_properties = {}
for pn in property_names:
    structure_properties[pn] = np.loadtxt(f'../Processed_Data/DEEM_330k/Data/structure_{pn}.dat')

# Linear Ridge Regression

In [7]:
# Use all defaults for the template parameters
ridge_parameters = dict()

# Regularization parameters for cross-validation
regularizations = np.logspace(-12, 0, 13)
parameter_grid = dict(ridge__regressor__alpha=regularizations)

In [9]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_330k/Data/{cutoff}'
    
    # Read SOAPs in training set
    soaps = utils.load_hdf5(f'{data_dir}/soaps_power_full_avg_nonorm.hdf5', indices=train_idxs)
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        property_label = pn.capitalize()
        
        # Set working directory
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LRR/{property_label}'
        os.makedirs(work_dir, exist_ok=True)
        
        # Load the property values (just from the train set)
        y = structure_properties[pn][train_idxs]
         
        # Cross validation pipeline
        cache_dir = mkdtemp(dir=work_dir)
        pipeline = Pipeline(
            [
                ('norm_scaler', utils.NormScaler()), 
                ('ridge', TransformedTargetRegressor(
                    regressor=Ridge(**ridge_parameters), 
                    transformer=utils.NormScaler()
                ))
            ],
            memory=cache_dir
        )
        gscv = GridSearchCV(
            pipeline, parameter_grid, 
            scoring=[
                'neg_root_mean_squared_error', 
                'neg_mean_absolute_error'
            ],
            cv=utils.cv_generator(cv_idxs),
            refit=False, return_train_score=True, error_score='raise'
        )
        gscv.fit(soaps, y)
        rmtree(cache_dir)
        
        save_json(gscv.cv_results_, f'{work_dir}/cv_results.json', array_convert=True)

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.97266e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.79976e-19): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.0104e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.35871e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.4765e-17): re

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.38302e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.37823e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.64105e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.36948e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.73214e-18): 

In [11]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LRR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}.json')

-----Optimal Parameters for 3.5 volumes mean_absolute_error-----
neg_mean_absolute_error = -2.5495712166987725
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -3.6876161795089772
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 energies mean_absolute_error-----
neg_mean_absolute_error = -0.6662672825771953
{'alpha': 1e-05}

-----Optimal Parameters for 3.5 energies root_mean_squared_error-----
neg_root_mean_squared_error = -0.9478438028357127
{'alpha': 0.001}

-----Optimal Parameters for 6.0 volumes mean_absolute_error-----
neg_mean_absolute_error = -1.0842773569634356
{'alpha': 0.01}

-----Optimal Parameters for 6.0 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -1.769226885522422
{'alpha': 0.01}

-----Optimal Parameters for 6.0 energies mean_absolute_error-----
neg_mean_absolute_error = -0.11706984695132286
{'alpha': 1e-07}

-----Optimal Parameters for 6.0 energies root_mean_squared_error-----


# Linear Ridge Regression (random train set)

In [12]:
# Train and test sets (random DEEM)
train_idxs_random = np.loadtxt('../Processed_Data/DEEM_330k/train_random.idxs', dtype=int)
cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_5.idxs', dtype=int)
sort_idxs = np.argsort(train_idxs_random)
rev_idxs = np.argsort(sort_idxs)

In [16]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_330k/Data/{cutoff}'
    
    # Read SOAPs in training set
    soaps = utils.load_hdf5(f'{data_dir}/soaps_power_full_avg_nonorm.hdf5', indices=train_idxs_random[sort_idxs])
    soaps = soaps[rev_idxs]
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        property_label = pn.capitalize()
        
        # Set working directory
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LRR/{property_label}'
        os.makedirs(work_dir, exist_ok=True)
        
        # Load the property values (just from the train set)
        y = structure_properties[pn][train_idxs_random]
         
        # Cross validation pipeline
        cache_dir = mkdtemp(dir=work_dir)
        pipeline = Pipeline(
            [
                ('norm_scaler', utils.NormScaler()), 
                ('ridge', TransformedTargetRegressor(
                    regressor=Ridge(**ridge_parameters), 
                    transformer=utils.NormScaler()
                ))
            ],
            memory=cache_dir
        )
        gscv = GridSearchCV(
            pipeline, parameter_grid, 
            scoring=[
                'neg_root_mean_squared_error', 
                'neg_mean_absolute_error'
            ],
            cv=utils.cv_generator(cv_idxs),
            refit=False, return_train_score=True, error_score='raise'
        )
        gscv.fit(soaps, y)
        rmtree(cache_dir)
        
        save_json(gscv.cv_results_, f'{work_dir}/cv_results_random.json', array_convert=True)

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.58211e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.95931e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.36968e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=2.16143e-18): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.23685e-17): 

/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.3952e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.3829e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.40187e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.39649e-17): result may not be accurate.
  overwrite_a=True).T
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.08835e-17): re

In [18]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Linear_Models/LRR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results_random.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}_random.json')

-----Optimal Parameters for 3.5 volumes mean_absolute_error-----
neg_mean_absolute_error = -2.627899363662225
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -3.7634674014467415
{'alpha': 0.001}

-----Optimal Parameters for 3.5 energies mean_absolute_error-----
neg_mean_absolute_error = -0.6785163484303577
{'alpha': 0.0001}

-----Optimal Parameters for 3.5 energies root_mean_squared_error-----
neg_root_mean_squared_error = -0.9651549966295478
{'alpha': 0.0001}

-----Optimal Parameters for 6.0 volumes mean_absolute_error-----
neg_mean_absolute_error = -1.1184560899853204
{'alpha': 0.01}

-----Optimal Parameters for 6.0 volumes root_mean_squared_error-----
neg_root_mean_squared_error = -1.774245524792089
{'alpha': 0.01}

-----Optimal Parameters for 6.0 energies mean_absolute_error-----
neg_mean_absolute_error = -0.11254803400456367
{'alpha': 1e-07}

-----Optimal Parameters for 6.0 energies root_mean_squared_error-----


# Kernel Ridge Regression

In [ ]:
# Use all defaults for the template parameters
kernel_ridge_parameters = dict(kernel='precomputed')

# Set ranges of kernel gamma (for Gaussian kernel) and regularization
log_gammas = np.linspace(-3, 3, 7)
##log_gammas = np.array([-2.0])

regularizations = np.logspace(-12, 0, 13)
##regularizations = np.array([1.0E-3, 1.0E-2])

# Use the filenames of the kernels that we will be loading
# in the "hacked" pipeline.
# The gamma names for the kernel parsing will be set in the loop
parameter_grid_base = dict(
    ridge__regressor__alpha=regularizations, 
)

In [ ]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_10k/Data/{cutoff}'
    model_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian'
    os.makedirs(model_dir, exist_ok=True)
    
    # Read SOAPs in training set
    soaps = utils.load_hdf5(f'{data_dir}/soaps_power_full_nonorm.hdf5') # Use this
    soaps = np.array([np.mean(soap, axis=0) for soap in soaps]) # Use this
    ##str_idxs = [str(i).zfill(4) for i in range(0, len(train_idxs))]
    ##soaps = utils.load_hdf5(f'{data_dir}/soaps.hdf5', datasets=str_idxs, concatenate=False)
    
    # Build a "superkernel": a concatenation of the train and test kernels
    # (which we can store easily in memory and on disk) so that
    # the pipeline doesn't compute whole new kernels for each CV set.
    # We still have to compute a kernel for each gamma, though.
    # We store the kernels named by the logarithm of the gamma parameter,
    # so the filenames don't get ridiculous
    ##XA = np.random.rand(10000, 1000)
    for log_gamma in tqdm(log_gammas, desc='Gamma', leave=False):
        gamma = 10 ** log_gamma
        K = gaussian_kernel(soaps, soaps, gamma=gamma) # Use this
        ##K = gaussian_kernel(XA, XA, gamma=gamma)
        utils.save_hdf5(
            f'{model_dir}/gaussian_kernel_{log_gamma}.hdf5', K, 
            attrs=dict(gamma=gamma, log_gamma=log_gamma), chunks=(100, 100)
        )

In [ ]:
# Hacked CV indices so we can load and access the kernels by indexing
# instead of recomputing them
dummy_cv_idxs = np.arange(0, cv_idxs.shape[-1]).reshape((1, -1))

In [ ]:
# Optimize hyperparameters

# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    # Set data directory
    model_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian'
    
    parameter_grid = deepcopy(parameter_grid_base)
    parameter_grid['kernel_loader__filename'] = \
        [f'{model_dir}/gaussian_kernel_{log_gamma}.hdf5' for log_gamma in log_gammas]
          
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        property_label = pn.capitalize()
        
        # Set working directory
        work_dir = f'{model_dir}/KRR/{property_label}'
        os.makedirs(work_dir, exist_ok=True)
                
        # Load the property values (just from the train set)
        y = structure_properties[pn][train_idxs]
        
        # "Hacked" pipeline: instead of recomputing the expensive kernels
        # at each CV iteration, we will fit instead with a set of indices
        # and use the filename containing the kernel at a particular
        # gamma as a hyperparameter for KernelLoader, which will
        # load the kernel at initialization.
                
        # NOTE: can't just use the custom kernel as a callable
        # to KernelRidge, as the custom kernel necessarily operates on 2D arrays
        # of the features for all environments in a given structure,
        # whereas the callable must operate on pairs of samples.
        # The KernelLoader/KernelConstructor is used instead
        cache_dir = mkdtemp(dir=work_dir)
        pipeline = Pipeline(
            [
                ('kernel_loader', utils.KernelLoader()),
                ('kernel_norm_scaler', utils.KernelNormScaler()), 
                ('ridge', TransformedTargetRegressor(
                    regressor=KernelRidge(**kernel_ridge_parameters),
                    transformer=utils.SampleSelector(X=y, model=utils.NormScaler()),
                    check_inverse=False
                ))
            ],
            memory=cache_dir
        )
        
        # "Hacked" CV: since we want to access the kernels via indices,
        # we fit the CV with cv_indices, and the cv_generator 
        # is used to pick out the correct folds
        gscv = GridSearchCV(
            pipeline, parameter_grid, 
            scoring=dict(
                neg_mean_absolute_error=make_scorer(
                    utils.score_by_index,
                    greater_is_better=False,
                    y=y, scorer=mean_absolute_error
                ),
                neg_root_mean_squared_error=make_scorer(
                    utils.score_by_index,
                    greater_is_better=False,
                    y=y, scorer=mean_squared_error,
                    squared=False
                )
            ),
            cv=utils.cv_generator(dummy_cv_idxs),
            refit=False, return_train_score=True, error_score='raise'
        )
        gscv.fit(cv_idxs.T, cv_idxs.T)
        save_json(gscv.cv_results_, f'{work_dir}/cv_results.json', array_convert=True)
        rmtree(cache_dir)

In [ ]:
# Extract optimal parameters as a check

# Loop over cutoffs
for cutoff in cutoffs:
    if cutoff != 3.5:
        continue
    
    # Loop over properties
    for pn in property_names:
        property_label = pn.capitalize()
        
        work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR/{property_label}'
        cv_results = load_json(f'{work_dir}/cv_results.json')
        
        # Loop over error types
        for error, error_name in zip(
            ['neg_mean_absolute_error', 'neg_root_mean_squared_error'],
            ['mae', 'rmse']
        ):
        
            idx = np.argmin(cv_results[f'rank_test_{error}'])
            opt_parameters = utils.get_optimal_parameters(cv_results, error, **kernel_ridge_parameters)

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error[4:]}-----')
            print(f'{error} =', cv_results[f'mean_test_{error}'][idx])
            print(opt_parameters)
            print('')
            
            # Save optimal parameters for easy access
            save_json(opt_parameters, f'{work_dir}/ridge_parameters_{error_name}.json')